In [0]:
!curl ifconfig.co

35.229.201.223


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls /content/gdrive/My\ Drive/
!cp -r /content/gdrive/My\ Drive/BO-* /content/gdrive/My\ Drive/resume
!cd gdrive/My\ Drive/colab
!unzip /content/gdrive/My\ Drive/colab/ml-100k.zip
!ls gdrive/My\ Drive/colab/ml-100k/

 BO-DA-رزومه‌ها-E0		  nimac.gsite
 colab				  resume
'Getting started.pdf'		  SahabX-Sokan
'hadoop meeting feedback.gform'  'اعضای دانشکده.xlsx'
'Hadoop Meeting.gslides'	 'پروژه هوش سحاب.gdoc'
 NimaBehrang.pdf		 'مشخصات سرور.gsheet'
Archive:  /content/gdrive/My Drive/colab/ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data           bad CRC d354e598  (should be 55baee78)
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  infla

In [2]:
!pip install -q tensorflow-gpu==2.0.0a0
!pip install -q tensorboard
!pip install -q tensorboardcolab

     |████████████████████████████████| 332.1MB 57kB/s 
     |████████████████████████████████| 419kB 52.4MB/s 
     |████████████████████████████████| 3.0MB 39.4MB/s 


In [3]:
from tensorflow.python import keras
import tensorflow as tf
from tensorboardcolab import *
tbc=TensorBoardColab(port=6006, graph_path='/tmp/logs', startup_waiting_time=8)

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
https://0e1910a6.ngrok.io


In [30]:
# !cat gdrive/My\ Drive/colab/ml-100k/u.user=
import numpy as np
data = np.genfromtxt("/content/gdrive/My Drive/colab/ml-100k/u.data", dtype=int)
train_data = []
test_data = []
UPL=10
K=20
users = np.unique(data[:,0])
for user in users:
  userrows = data[(np.where(data[:,0] == user)), :][0]
  if len(userrows) > UPL + K:
    np.random.shuffle(userrows)
    for item in userrows[:UPL]:
      train_data.append(item)
    for item in userrows[UPL:]:
      test_data.append(item)
train_data = np.asarray(train_data)
test_data = np.asarray(test_data)

print(len(train_data))
print(np.shape(train_data))
print(train_data[0])

7300
(7300, 4)
[        1        88         4 878542791]


In [0]:
def castRate(inp):
  return inp/5-0.1

In [0]:
keras.backend.clear_session()

In [0]:
# input_user = keras.Input(shape=(1,))  # Returns a placeholder tensor
# input_item = keras.Input(shape=(1,))  # Returns a placeholder tensor
# embed_user = keras.layers.Embedding(2000, 64)(input_user)
# embed_item = keras.layers.Embedding(2000, 64)(input_item)
# multiply = keras.layers.Multiply()([embed_user, embed_item])
# rate = keras.layers.add(tf.unstack(multiply, axis=2))
# model = keras.Model(inputs=[input_user, input_item], outputs = keras.activations.sigmoid(rate))
# model.summary()

In [22]:
input_user = keras.Input(shape=(1,))  # Returns a placeholder tensor
input_item = keras.Input(shape=(1,))  # Returns a placeholder tensor
embed_user = keras.layers.Embedding(2000, 64)(input_user)
embed_item = keras.layers.Embedding(2000, 64)(input_item)
dot = keras.layers.dot(inputs=[embed_user, embed_item],axes=2)
model_opt = keras.Model(inputs=[input_user, input_item], outputs = keras.activations.sigmoid(dot))
model_opt.summary()
sgd = tf.keras.optimizers.SGD(lr=0.01, clipvalue=0.5)
adam = tf.keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
nadam = tf.keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
square = tf.keras.losses.squared_hinge
ms = 'mean_squared_error'
model_opt.compile(optimizer=adam,
            loss=ms,
            metrics=['accuracy','mae'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 64)        128000      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 64)        128000      input_2[0][0]                    
______________________________________________________________________________________________

In [35]:
x_train = [train_data[:,0],train_data[:,1]]
y_train = [castRate(x) for x in train_data[:,2]]
x_test = [test_data[:,0],test_data[:,1]]
y_test = [castRate(x) for x in test_data[:,2]]

logdir="/tmp/logs/2"
checkpoint_path = logdir + "/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=checkpoint_path, histogram_freq=0,
                                             write_graph=True, write_grads=False, write_images=False,
                                             embeddings_freq=0, embeddings_layer_names=None,
                                             embeddings_metadata=None, embeddings_data=[], update_freq='epoch')
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=1, verbose=1,
                                               mode='auto', baseline=None, restore_best_weights=False)

fit = model_opt.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=30,
                callbacks = [cp_callback, tb_callback, es_callback], batch_size=1000)

# ans = model_opt.evaluate(x_test, y_test)

W0729 12:40:45.113296 140224684017536 tf_logging.py:161] Model failed to serialize as JSON. Ignoring... ('Not JSON Serializable:', b'\n\tSigmoid_1\x12\x07Sigmoid\x1a\ndot/MatMul*\x07\n\x01T\x12\x020\x01')


Train on 7300 samples, validate on 87549 samples
Epoch 1/30
1000/7300 [===>..........................] - ETA: 0s - loss: 0.0623 - accuracy: 0.0000e+00 - mae: 0.2001
Epoch 00001: saving model to /tmp/logs/2/cp.ckpt
7300/7300 [==============================] - 0s 42us/sample - loss: 0.0663 - accuracy: 0.0000e+00 - mae: 0.2067 - val_loss: 0.0587 - val_accuracy: 0.0000e+00 - val_mae: 0.1983
Epoch 2/30
1000/7300 [===>..........................] - ETA: 0s - loss: 0.0616 - accuracy: 0.0000e+00 - mae: 0.1992
Epoch 00002: saving model to /tmp/logs/2/cp.ckpt
7300/7300 [==============================] - 0s 42us/sample - loss: 0.0669 - accuracy: 0.0000e+00 - mae: 0.2071 - val_loss: 0.0588 - val_accuracy: 0.0000e+00 - val_mae: 0.1982
Epoch 00002: early stopping


1.0991902834008087


In [60]:
def calcHitRatio(test_data):
  x_test = [test_data[:,0],test_data[:,1]]
  y_test = [castRate(x) for x in test_data[:,2]]
  predict_rate = model_opt.predict(x_test)
  users_number = np.unique(test_data[:,0])
  predict = [(user, movie, rate[0][0]) for user,movie,rate in zip(test_data[:,0], test_data[:,1],predict_rate)]
  real = [(user, movie, castRate(rate)) for user,movie,rate in zip(test_data[:,0], test_data[:,1], test_data[:,2])]

  print(predict[:20])
  print(real[:20])

  avr, num = (0,0)
  for selected_user in users_number:
    predict_for_user = sorted([(user, movie, rate) for user,movie,rate in predict if user == selected_user], key=lambda x: x[2], reverse=True)
    real_for_user = sorted([(user, movie, rate) for user,movie,rate in real if user == selected_user], key=lambda x: x[2], reverse=True)
    count = 0
    if selected_user == 1:
        print(predict_for_user)
        print(real_for_user)
    for u,m,r in real_for_user[:K]:
      count += len([x for x,y,z in predict_for_user[:K] if y == m])
    if selected_user < 10:
      print(count)
    avr = (avr*num + count)/(num+1)
    num = num+1
  return (avr,num)


  # ans = []
  # for indx, item in enumerate(predict):
  #   ans.append([indx, item])
  # ans.sort(key=lambda x: x[1],reverse=True)
  # tresh = 100
  # count = 0
  # for i in range(tresh):
  #     if ans[i][0] <= tresh:
  #       count = count + 1
  # return count
print(calcHitRatio(test_data))


[(1, 2, 0.74962246), (1, 17, 0.77996224), (1, 222, 0.757692), (1, 201, 0.78026235), (1, 13, 0.7386266), (1, 125, 0.5979352), (1, 47, 0.68791056), (1, 120, 0.8903676), (1, 92, 0.6804207), (1, 180, 0.4366649), (1, 73, 0.7684903), (1, 134, 0.55385995), (1, 257, 0.5791896), (1, 169, 0.44267735), (1, 188, 0.52601796), (1, 27, 0.5989295), (1, 123, 0.6620594), (1, 80, 0.6913945), (1, 227, 0.7371909), (1, 86, 0.5521065)]
[(1, 2, 0.5), (1, 17, 0.5), (1, 222, 0.7000000000000001), (1, 201, 0.5), (1, 13, 0.9), (1, 125, 0.5), (1, 47, 0.7000000000000001), (1, 120, 0.1), (1, 92, 0.5), (1, 180, 0.5), (1, 73, 0.5), (1, 134, 0.7000000000000001), (1, 257, 0.7000000000000001), (1, 169, 0.9), (1, 188, 0.5), (1, 27, 0.30000000000000004), (1, 123, 0.7000000000000001), (1, 80, 0.7000000000000001), (1, 227, 0.7000000000000001), (1, 86, 0.9)]
[(1, 101, 0.89621246), (1, 109, 0.89060336), (1, 120, 0.8903676), (1, 145, 0.88137746), (1, 139, 0.87959534), (1, 196, 0.879302), (1, 171, 0.8721964), (1, 244, 0.8459172),